# Analiza zadnjih 100 oglasov na portalu avto.net (dne 26.10.2023)

Na portalu [avto.net](https://www.avto.net/) je posebna [stran](https://www.avto.net/Ads/results_100.asp?oglasrubrika=1&prodajalec=2), ki vsebuje zadnjih 100 objavljenih oglasov (stran se sproti spreminja).



In [6]:
#s kodo v tem okvirčku izluščimo html iz spletne podstrani, ki vsebuje zadnjih 100 oglasov
import requests
from bs4 import BeautifulSoup

headers = {
    'Accept': 'text/html, image/avif, image/webp, image/apng, image/svg+xml, */ *; q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US, en;q=0.9',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Site': 'none',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

data_request = requests.get('https://www.avto.net/Ads/results_100.asp?', headers=headers)
soup = BeautifulSoup(data_request.content, 'html.parser')
vsebina_html = soup.prettify()
with open("zadnjih_100.html", "w", encoding='utf-8') as f:
     print(vsebina_html, file=f)


# print(data_request.status_code)

NameError: name 'requests' is not defined

In [42]:
#s kodo v tem okvirčku bomo iz datoteke zadnjih_100.html izluščili podatke (ime, 1.registracija, prevoženih km, gorivo, menjalnik, motor)
#in jih zapisali v slovar oblike {'ime avta':{'1. registracija': 2011, 'Prevoženih': '270000 km', 'Gorivo': 'bencinski motor', 'Menjalnik': 'avtomatski menjalnik', 'Moč motorja': '160 KM'}}
#slovar sem izbral zato, ker so se določeni podatki v kodi dvakrat ponovili. Nekateri avti pa tudi nimajo podatka o npr. prevoženih kilometrih. Pri nekaterih električnih avtih
#pa pride podatek tipa 'Baterija': '52 kw', ki ga v slovar nisem zapisal, moč motorja pa sem iz kW pretvoril v KM (1 kW = 1.341 KM) in zaokrožil na najbližje celo število.

from bs4 import BeautifulSoup

with open('zadnjih_100.html', 'r', encoding='utf-8') as f:
    vsebina = f.read()
    soup = BeautifulSoup(vsebina, 'html.parser')

    #Pridelamo seznam z imeni vseh avtov (kot so ta zapisana na avto.net)

    imena_avtov = []

    #Ker imajo vsa imena avtov pred njimi značko <div class="GO-Results-Naziv ... to izkoristimo, da izluščimo imena ven
    for element in soup.find_all(class_='GO-Results-Naziv'):
        imena_avtov.append(element.text.strip())


    #Sedaj pa pridelamo še en seznam, z vsemi drugimi podatki (hočem, da mi pri motorju pobere samo konjske moči)

    drugi_podatki = []
    for element in soup.find_all('td', class_='pl-3'):
        drugi_podatki.append(element.text.strip())


    #hočem, da mi pri motorju pobere samo konjske moči, morebiten podatek samo v kW mi pretvori v KM (1 kW = 1.341 KM) in zaokroži na najbližje celo število

    for i in range(len(drugi_podatki)):
        if '/' in drugi_podatki[i]:
            drugi_podatki[i] = drugi_podatki[i][(-6):]
        if drugi_podatki[i][0] == ' ':
            drugi_podatki[i] = drugi_podatki[i][1:]
   
    for i in range(len(drugi_podatki)):
        if drugi_podatki[i][(-2):] == 'kW':
            drugi_podatki[i] = drugi_podatki[i][:(-2)]
            drugi_podatki[i] = str(round(int(drugi_podatki[i]) * 1.341)) + ' KM'

    #za vsako ime avta bo treba zgenerirat svoj vmesni slovar

    def na_katerih_indeksih_se_nahaja(sez, a):
        i = 0
        sez_indeksov = []
        for el in sez:
            if el == a:
                sez_indeksov.append(i)
            i += 1
        return sez_indeksov
    
    #koda za izdelavo slovarja
    sez_indeksov = na_katerih_indeksih_se_nahaja(drugi_podatki, '1.registracija')
    slovar = {}
    vmesni_slovar = {}
    seznam_kljucev = ['1.registracija', 'Prevoženih', 'Gorivo', 'Menjalnik', 'Motor']
    i = 1
    j = 0
    for el in drugi_podatki:
        if i in sez_indeksov: #natanko tedaj, ko naletimo na prvo registracijo, hočemo začet nov vmesni slovar
            slovar[imena_avtov[j]] = vmesni_slovar
            vmesni_slovar = {}
            j += 1
        if drugi_podatki[i - 1] in seznam_kljucev:
            vmesni_slovar[drugi_podatki[i - 1]] = drugi_podatki[i]
        i += 1
            
    print(slovar)

{'Volkswagen Passat CC 1.8 TSI avt.': {'1.registracija': '2011', 'Prevoženih': '270000 km', 'Gorivo': 'bencinski motor', 'Menjalnik': 'avtomatski menjalnik', 'Motor': '160 KM'}, 'Kia Ceed': {'1.registracija': '2008', 'Prevoženih': '200000 km', 'Gorivo': 'bencinski motor', 'Menjalnik': 'ročni menjalnik', 'Motor': '109 KM'}, 'Opel Insignia 2.0 CDTI EDITION VELIKA NAVI-PDC-TEMP.-ALU-XENON-': {'1.registracija': '2015', 'Prevoženih': '241031 km', 'Gorivo': 'diesel motor', 'Menjalnik': 'ročni menjalnik', 'Motor': '163 KM'}, 'Renault Clio Rip Curl2 1.2 16v 3V': {'1.registracija': '2008', 'Prevoženih': '263000 km', 'Gorivo': 'bencinski motor', 'Menjalnik': 'ročni menjalnik', 'Motor': '75 KM'}, 'Opel Grandland X 1.5 DIESEL 130 Elite - BOGATO OPREMLJEN': {'1.registracija': '2020', 'Prevoženih': '140000 km', 'Gorivo': 'diesel motor', 'Menjalnik': 'ročni menjalnik', 'Motor': '131 KM'}, 'Audi A4 Avant 2.0 TDI S tronic 110  150': {'1.registracija': '2016', 'Prevoženih': '270000 km', 'Gorivo': 'diese